<a href="https://colab.research.google.com/github/shauryasingh000/Mushroom_Class_prediction_using_MachineLearning/blob/main/Mushroom_Class_prediction_ML_(99_Accuracy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'mushroom-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F478%2F974%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240905%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240905T060807Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0944630cc4f5eb1d6b195960beb88ea5bb857c563b11bbd1635413a06f77fa272455526977d7f93f07a84969cda84089cab9278fff792b41ed837e784ddbb537a1720c9ea051751867faae9183453896dbddb392061062dfcbfa8c6e54d2a1b044263735f6d35ad33259ea6776fcc20cfe86dc03af707b62d535d9f67d16ffd6e1ad9c7b70acb428d0892209c6e1ae7913e8d6eda544fa38e8786c83116746524673c7df32309c03a8eef01711f8f0f04b367571012293ae413057b2140fec13f1d8a8a3009b1f1bdfe4d43a785968a68178ed191d08df1f1294ea39329e116da2df7d88f5cdd3fec0bfb91d5b6f162cea3724ffe656d7639594dba307a64006'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

In [ ]:
data = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
data.sample(10)

In [ ]:
data.info()

**Data mostly in string format need to be converted into numerical col to perform further function**

In [ ]:
data.describe().T

In [ ]:
data.isnull().sum()

Clean Data

In [ ]:
# Encoding the data

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in data.columns:
    data[col] = le.fit_transform(data[col])

data

# Basic Data Visualization

In [ ]:
data_corr = data.corr()

plt.figure(figsize=(15,15))
sns.heatmap(data_corr,annot =True)

In [ ]:
px.box(data,color ='class')

In [ ]:
data.hist(figsize = (15,15))

In [ ]:
sns.pairplot(data,hue='class')

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['class'])
y = data['class']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2 , random_state =42)
X_train

**This Data is a Classification type Dataset**

In [ ]:
# Importing suitable Libraries for this purpose
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier



In [ ]:
from sklearn.metrics import accuracy_score,classification_report,r2_score
from sklearn import metrics


**KNN CLASSIFIER**

In [ ]:
knn = KNeighborsClassifier()

X_train_knn = X_train.copy(deep = True)
y_train_knn = y_train.copy(deep = True)

param_grid = {'n_neighbors': range(1,11),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']}
gv = GridSearchCV(knn,param_grid = param_grid,cv = 4,scoring = 'accuracy')

gv.fit(X_train_knn,y_train_knn)

best_model = gv.best_estimator_
best_parameter = gv.best_params_

display(best_model,best_parameter)

In [ ]:
knn = KNeighborsClassifier(metric='manhattan', n_neighbors =  1, weights = 'uniform')

knn.fit(X_train_knn,y_train_knn)
pred_knn = knn.predict(X_test)

print(classification_report(y_test,pred_knn))



In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_knn)
roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='KNN Classifier',)

display.plot()
plt.show()

In [ ]:
svc = SVC()

X_train_svc = X_train.copy(deep = True)
y_train_svc = y_train.copy(deep = True)


param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['auto', 'scale'],
    'class_weight': ['balanced', None]}

gv = GridSearchCV(svc,param_grid = param_grid,cv = 4,scoring = 'accuracy')

gv.fit(X_train_svc,y_train_svc)

best_model = gv.best_estimator_
best_parameter = gv.best_params_

display(best_model,best_parameter)

In [ ]:
svc = SVC(C = 1,class_weight ='balanced',gamma='auto',kernel = 'rbf')

svc.fit(X_train_svc,y_train_svc)
pred_svc = svc.predict(X_test)

print(classification_report(y_test,pred_svc))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_svc)
roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='SVC',)

display.plot()
plt.show()

In [ ]:
dt = DecisionTreeClassifier()

X_train_dt = X_train.copy(deep = True)
y_train_dt = y_train.copy(deep = True)


param_grid = {'max_depth': [3, 5, 7, 10, 12],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy']}

gv = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy')

gv.fit(X_train_dt,y_train_dt)

best_model = gv.best_estimator_
best_parameter = gv.best_params_

display(best_model,best_parameter)


In [ ]:
dt  = DecisionTreeClassifier(criterion = 'gini',max_depth = 12,min_samples_leaf =1,min_samples_split = 2)

dt.fit(X_train_dt,y_train_dt)
pred_dt = dt.predict(X_test)

print(classification_report(y_test,pred_dt))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_dt)
roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='KNN Classifier',)

display.plot()
plt.show()

In [ ]:
rf = RandomForestClassifier()

X_train_rf = X_train.copy(deep = True)
y_train_rf = y_train.copy(deep = True)

param_grid = {'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5],
    'max_features': ['auto', 'sqrt', 'log2'],}

gv = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

gv.fit(X_train_rf,y_train_rf)

best_model = gv.best_estimator_
best_parameter = gv.best_params_

display(best_model,best_parameter)

In [ ]:
rf = RandomForestClassifier(n_estimators =100,
                           max_depth=10,
                           min_samples_split =2,
                           min_samples_leaf=1,
                           max_features='auto')

rf.fit(X_train_dt,y_train_dt)
pred_rf = dt.predict(X_test)

print(classification_report(y_test,pred_rf))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_rf)
roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='Random Forest Classifier',)

display.plot()
plt.show()